In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list, p="1y"):
    return yf.download(tickers_list, period=p)

In [9]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*********************100%%**********************]  7346 of 7346 completed


55 Failed downloads:
['IGTAR', 'CETUR', 'LBBBR', 'YOTAR', 'FLFVR', 'GLLIR', 'AQUNR', 'ADOCR', 'EFHTR', 'WTMAR', 'ENERR', 'MARXR', 'CSLMR', 'AIBBR', 'ASCAR', 'FEXDR', 'ATMVR', 'ONFOW', 'SAGAR', 'BWAQR', 'BUJAR', 'BRLIR', 'DISTR', 'NOVVR', 'JGGCR', 'KACLR', 'PLTNR', 'IMAQR', 'NVACR', 'STRCW', 'HHGCR', 'BREZR', 'GECCZ', 'GBBKR', 'TMTCR', 'DMAQR', 'CLOER', 'ATMCR', 'CLRCR', 'PPHPR', 'ATAKR', 'ARIZR', 'GLSTR', 'MCACR', 'GODNR', 'BLEUR', 'BNIXR', 'ASCBR', 'RWODR', 'QOMOR', 'GDSTR', 'BRACR', 'TENKR', 'MBTCR']: Exception("%ticker%: Period '1y' is invalid, must be one of ['1d', '5d']")
['EMP']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')



313.6618163585663


In [14]:
df = get_info(data)
cop = df.copy()
# df

### Filtering out stocks that don't match 

In [15]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 1000
vold_thresh = 1000000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=200)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(15)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
ARGO,29.83,30.13,1.01,"55,826.00",2023-04-06,"1,665,010.49"
CIR,55.71,56.48,1.37,"117,399.00",2023-06-30,"6,539,711.51"
CNK,18.43,18.85,2.20,"847,622.00",2023-06-13,"15,625,911.12"
KAI,224.61,229.90,2.30,"17,136.00",2023-06-16,"3,848,916.97"
HSPO,10.58,10.84,2.35,"123,950.00",2023-06-05,"1,311,390.99"
HCVI,10.34,10.60,2.45,"913,062.00",2023-06-16,"9,441,061.22"
SWBI,13.15,13.69,3.91,"84,761.00",2023-06-26,"1,115,030.93"
GAQ,10.76,11.20,3.93,"200,097.00",2023-05-24,"2,153,043.77"
MANH,198.31,208.31,4.80,"67,468.00",2023-07-20,"13,379,578.92"
BDC,94.20,99.13,4.97,"32,802.00",2023-07-18,"3,089,948.30"


# Testing out (manually backtesting)

In [32]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [33]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,127.62,160.26,20.37,"1,333,326.33",2022-12-13,126.87,20.83,160.26,2022-12-13
AA,33.38,58.24,42.68,"3,132,267.33",2022-08-26,33.80,41.96,58.24,2022-08-26
AAC,10.56,10.60,0.38,"21,163.33",2023-07-18,10.65,0.05,10.65,2023-07-31
AACG,1.39,2.64,47.35,"2,835.00",2022-08-18,1.32,50.00,2.64,2022-08-18
AACI,10.51,12.20,13.85,"8,276.50",2023-01-30,NaN,NaN,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.20,25.00,99.21,"1,000,905.00",2022-12-12,0.17,99.33,25.00,2022-12-12
ZWS,27.33,31.77,13.98,"590,255.00",2022-08-18,30.21,4.91,31.77,2022-08-18
ZYME,7.87,10.80,27.13,"923,709.67",2023-01-19,7.16,33.66,10.80,2023-01-19
ZYNE,0.36,1.39,74.10,"96,968.00",2022-08-08,0.35,74.97,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [8]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [6]:
def parse_df(df, old_high=2, high_by="High"):
    # old_high- How many days ago it broke high (x or less)
    # high_by- by which OHLC to determine d
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})

    return new_df

### Filtering and sorting DataFrame

In [7]:
def filter_sort_df(df):
    df = df.copy()
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=50)# Hit the high between 120 and 200 days ago (3 - 7 months)
    # time_thresh1_b = datetime.now() - timedelta(days=60) 
    time_thresh2 = datetime.now() - timedelta(days=245)  
    # time_thresh2_b = datetime.now() - timedelta(days=200)
    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of high"] != df["Date of high"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [8]:
full_df = parse_df(data, 2)
cop = full_df.copy()

df = filter_sort_df(cop)
df

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$
PNT,12.40,12.41,0.08,"9,434,100.00",2023-10-03,2023-05-15,"116,982,836.40"
DSGR,27.63,28.66,3.59,"66,400.00",2023-10-04,2023-07-31,"1,834,631.94"
NUVL,57.51,61.70,6.79,"6,240,500.00",2023-10-04,2023-08-01,"358,891,144.53"
IRRX,10.92,11.87,8.00,"354,100.00",2023-10-04,2023-08-14,"3,866,772.03"


In [13]:
add_to_tv(df)

['CASY', 'EC', 'BBCP', 'DO', 'KRYS', 'SFWL']


In [14]:
cop.loc["URI"]

Close                            449.41
52wk High                        492.22
% to 52wk                          8.70
Volume                        24,350.00
Date of high        2023-08-07 00:00:00
Old date of high    2023-08-07 00:00:00
Volume$                   10,943,133.59
Name: URI, dtype: object

# Finding low 52

In [6]:
def parse_df_low(df, old_low=2, low_by="Low"):
    # old_low- How many days ago it broke low (x or less)
    # low_by- by which OHLC to determine d
    low = df['Low'].min()  # 52 wk low
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((low - close) / low) * 100  # Percent change to 52wk low
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['Low'][:-old_low].idxmin()  # 52wk low before 2 days
    doh = df['Low'].idxmin()  # 52wk low today
    new_df = pd.DataFrame({'Close': close, '52wk Low': low, "% to 52wk": pct_to_52, "Volume": volume, "Date of low": doh, "Old date of low": old_doh, "Volume$": volume * close})

    return new_df

def filter_sort_df_low(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a low volume but due to low price they have a low volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the low between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of low"] < time_thresh1) & (df["Old date of low"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of low"] != df["Date of low"]]
    
    # return df.sort_values(by='% to 52wk', ascending=True)
    return df.sort_values(by='Date of low', ascending=True)

In [7]:
full_df = parse_df_low(data, 12)
cop = full_df.copy()

df = filter_sort_df_low(cop)
df

,Close,52wk Low,% to 52wk,Volume,Date of low,Old date of low,Volume$
AHT,2.68,2.58,-3.88,"344,800.00",2023-08-22,2023-03-24,"924,064.02"
LC,6.85,6.23,-9.95,"1,428,700.00",2023-08-22,2023-05-04,"9,786,594.86"
SHBI,10.83,10.61,-2.07,"239,900.00",2023-08-22,2023-05-12,"2,598,116.98"
AROW,17.58,16.88,-4.15,"97,200.00",2023-08-23,2023-06-01,"1,708,775.99"
PTON,5.82,5.05,-15.25,"8,151,200.00",2023-08-23,2023-05-12,"47,439,985.40"
...,...,...,...,...,...,...,...
NXST,133.65,133.32,-0.25,"1,131,200.00",2023-09-07,2023-05-31,"151,184,873.10"
DG,123.72,123.31,-0.33,"5,281,400.00",2023-09-07,2023-06-12,"653,414,814.45"
OPI,5.85,5.81,-0.69,"2,365,600.00",2023-09-07,2023-05-02,"13,838,759.77"
EDBL,0.89,0.77,-15.89,"4,558,500.00",2023-09-07,2023-06-20,"4,057,064.93"


In [26]:
copcop = data.copy()
copcop = copcop.bfill()
copcop["Close"]

,A,AA,AAC,AACG,AACI,AACIU,AACIW,AACT,AADI,AAIC,...,ZUO,ZURA,ZURAW,ZVIA,ZVRA,ZVSA,ZWS,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-02,129.30,47.91,9.90,2.22,9.89,9.96,0.08,10.11,13.26,3.18,...,7.94,7.24,0.45,4.72,5.56,9.98,26.99,4.76,1.11,8.78
2022-09-06,129.30,47.91,9.90,2.22,9.89,9.96,0.08,10.11,13.26,3.18,...,7.89,7.24,0.45,4.72,5.56,9.98,26.99,4.76,1.11,8.78
2022-09-07,131.43,47.66,9.92,2.16,9.89,9.96,0.08,10.11,13.07,3.18,...,7.85,7.24,0.45,4.89,5.65,9.98,27.65,4.65,1.12,9.17
2022-09-08,135.19,48.86,9.92,2.15,9.89,9.96,0.08,10.11,12.87,3.20,...,7.92,7.24,0.45,5.05,5.93,9.98,27.98,5.07,1.18,9.30
2022-09-09,137.63,52.62,9.91,2.04,9.89,9.96,0.08,10.11,12.54,3.19,...,8.33,7.24,0.45,4.94,6.18,10.00,28.55,5.22,1.19,9.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-28,119.92,28.48,10.69,1.21,10.61,10.65,0.08,10.21,6.80,4.57,...,8.88,6.73,0.45,2.34,5.11,0.15,29.23,7.49,1.32,7.66
2023-08-29,122.02,29.61,10.69,1.24,10.61,10.65,0.08,10.21,6.71,4.62,...,8.91,6.79,0.45,2.49,5.31,0.14,29.39,7.53,1.32,7.71
2023-08-30,122.68,29.51,10.70,1.26,10.53,10.65,0.08,10.23,6.68,4.61,...,9.05,6.91,0.45,2.59,5.30,0.14,29.57,7.35,1.32,7.75
